В данном ноутбуке использована архитектура BERT - предобученная модель, продемонстрировавшая высокую эффективность в решении ряда NLP-задач. В рамках нашего курса модель не рассматривалась, вот подборка ссылок на ресурсы для ознакомления с темой:

* Оригинальные статьи
    * [Ilya Sutskever. Sequence to Sequence Learning with Neural Networks](https://arxiv.org/abs/1409.3215)
    * [Dzmitry Bahdanau. Neural Machine Translation by Jointly Learning to Align and Translate](https://arxiv.org/abs/1409.0473)
    * [Ashish Vaswani. Attention Is All You Need](https://arxiv.org/abs/1706.03762)
    * [Jacob Devlin. BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding](https://arxiv.org/abs/1810.04805)
* Дополнительные материалы
    * [Jay Alammar. Visualizing A Neural Machine Translation Model (Mechanics of Seq2seq Models With Attention)](https://jalammar.github.io/visualizing-neural-machine-translation-mechanics-of-seq2seq-models-with-attention/)
    * [Jay Alammar. The Illustrated Transformer](https://jalammar.github.io/illustrated-transformer/)
    * [Chaitanya Joshi. Transformers are Graph Neural Networks](https://graphdeeplearning.github.io/post/transformers-are-gnns/)
    * [Jay Alammar. The Illustrated BERT, ELMo, and co. (How NLP Cracked Transfer Learning)](https://jalammar.github.io/illustrated-bert/)
<br><br>
---
В качестве шаблона при разработке ноутбука был использован [демо-пример по работе с BERT с официального сайта tensorflow](https://www.tensorflow.org/text/tutorials/bert_glue).

Модель содержит большое число параметров и требовательна к ресурсам, что обуславливает необходимость использования тензорного процессора (TPU) в Google Colab. Переход на TPU доступен через главное меню Colab: Runtime -> Change runtime type

In [1]:
!pip install -q -U tensorflow-text
!pip install -q -U tf-models-official
!pip install -q -U tfds-nightly

     |████████████████████████████████| 4.9 MB 5.6 MB/s 
     |████████████████████████████████| 1.8 MB 5.3 MB/s 
     |████████████████████████████████| 596 kB 51.8 MB/s 
     |████████████████████████████████| 352 kB 49.7 MB/s 
     |████████████████████████████████| 43 kB 1.4 MB/s 
     |████████████████████████████████| 47.7 MB 5.9 MB/s 
     |████████████████████████████████| 1.1 MB 48.4 MB/s 
     |████████████████████████████████| 90 kB 7.9 MB/s 
     |████████████████████████████████| 1.2 MB 52.8 MB/s 
     |████████████████████████████████| 213 kB 52.5 MB/s 
     |████████████████████████████████| 99 kB 6.6 MB/s 
     |████████████████████████████████| 4.1 MB 5.7 MB/s 


In [2]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import tensorflow_text as text
import tensorflow_addons as tfa

from official.nlp import optimization

In [3]:
tf.get_logger().setLevel('ERROR')

#необходимо загружать модели в распакованном виде, поскольку при использовании TPU нет 
#возможности загрузить архив и сохранить его на локальный диск в распакованном виде
os.environ["TFHUB_MODEL_LOAD_FORMAT"]="UNCOMPRESSED"

# модель состоит из двух блоков - препроцессора и энкодера, которые могут быть загружены из 
# репозитория tfhub по указанным ссылкам
preprocessor_name = 'https://tfhub.dev/tensorflow/albert_en_preprocess/3'
encoder_name = 'https://tfhub.dev/tensorflow/albert_en_xxlarge/3'

Подключение к TPU

In [4]:
cluster_resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
tf.config.experimental_connect_to_cluster(cluster_resolver)
tf.tpu.experimental.initialize_tpu_system(cluster_resolver)
strategy = tf.distribute.TPUStrategy(cluster_resolver)
print('Using TPU')

Using TPU


Загрузка данных

In [6]:
path = 'https://raw.githubusercontent.com/chekhovana/courses/main/machine_learning/6_final_project/3_kaggle_competition/data/'
path = os.path.join(path, 'products_sentiment_{dataset}.tsv')
fname_train, fname_test = (path.format(dataset=ds) for ds in ('train', 'test'))

data = pd.read_csv(fname_train, sep='\t', header=None)
train_size = 1600
valid_size = data.shape[0] - train_size
train_data = data[:train_size]
valid_data = data[-valid_size:]
print(train_data.shape, valid_data.shape)

batch_size = 32

(1600, 2) (400, 2)


Функция для создания препроцессора. Он будет приводить входные данные к тому формату, в котором они должны подаваться на вход энкодера

In [ ]:
def create_preprocessor(name, seq_length=128):
    input_segments = [tf.keras.layers.Input(shape=(), dtype=tf.string, 
                                            name='sentence')]
    preprocessor = hub.load(name)
    tokenizer = hub.KerasLayer(preprocessor.tokenize, name='tokenizer')
    segments = [tokenizer(s) for s in input_segments]

    packer = hub.KerasLayer(preprocessor.bert_pack_inputs,
                            arguments=dict(seq_length=seq_length),
                            name='packer')
    model_inputs = packer(segments)
    return tf.keras.Model(input_segments, model_inputs)

Функция для создания датасета, преобразованного к требуемому энкодером формату. Для переформатирования данных используется передаваемый в качестве аргумента препроцессор

In [ ]:
def create_dataset(data, preprocessor, is_train=True):
    dataset = tf.data.Dataset.from_tensor_slices(
        {'sentence': data[0].values, 'label': data[1].values})
    if (is_train):
        dataset = dataset.shuffle(data.shape[0])
        dataset = dataset.repeat()
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(lambda ex: (preprocessor(ex), ex['label']))
    dataset = dataset.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
    return dataset

Функция для создания энкодера. В качестве параметра передается ссылка на предобученную модель.

In [ ]:
def create_encoder(name):

  class Classifier(tf.keras.Model):
    def __init__(self):
        super(Classifier, self).__init__(name="prediction")
        self.encoder = hub.KerasLayer(name, trainable=True)
        self.dropout = tf.keras.layers.Dropout(0.1)
        self.dense = tf.keras.layers.Dense(2)

    def call(self, preprocessed_text):
      encoder_outputs = self.encoder(preprocessed_text)
      pooled_output = encoder_outputs["pooled_output"]
      x = self.dropout(pooled_output)
      x = self.dense(x)
      return x

  return Classifier()

Обучение модели.

In [ ]:
%%time

batch_size = 32
init_lr = 2e-5
epochs = 20

steps_per_epoch = train_size // batch_size
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = num_train_steps // 10
validation_steps = valid_size // batch_size

optimizer = optimization.create_optimizer(
    init_lr=init_lr,
    num_train_steps=num_train_steps,
    num_warmup_steps=num_warmup_steps,
    optimizer_type='adamw')

#для предотвращения переобучения будем использовать callback, запоминающий веса модели на 
#том шаге, которому соответствовала наилучшая точность прогноза на проверочной выборке
callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy', restore_best_weights=True, patience=5)

preprocessor = create_preprocessor(preprocessor_name)
train_dataset = create_dataset(train_data, preprocessor, is_train=True)
valid_dataset = create_dataset(valid_data, preprocessor, is_train=False)

with strategy.scope():
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    metric = tf.keras.metrics.SparseCategoricalAccuracy(
        'accuracy', dtype=tf.float32)


    model = create_encoder(encoder_name)
    model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

    history = model.fit(
        x=train_dataset, validation_data=valid_dataset,
        steps_per_epoch=steps_per_epoch, epochs=epochs, callbacks=[callback],
        validation_steps=validation_steps)

    print(model.evaluate(valid_dataset))

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:585: UserWarning: Input dict contained keys ['label'] which did not match any model input. They will be ignored by the model.
  [n for n in tensors.keys() if n not in ref_input_names])


Epoch 1/20


/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("AdamWeightDecay/gradients/PartitionedCall:1", shape=(None,), dtype=int32), values=Tensor("clip_by_global_norm/clip_by_global_norm/_0:0", shape=(None, 128), dtype=float32), dense_shape=Tensor("AdamWeightDecay/gradients/PartitionedCall:2", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)


50/50 [==============================] - 77s 708ms/step - loss: 0.6463 - accuracy: 0.7100 - val_loss: 0.3116 - val_accuracy: 0.8932
Epoch 2/20
50/50 [==============================] - 27s 548ms/step - loss: 0.2119 - accuracy: 0.9444 - val_loss: 0.3436 - val_accuracy: 0.9036
Epoch 3/20
50/50 [==============================] - 27s 547ms/step - loss: 0.1567 - accuracy: 0.9613 - val_loss: 0.3155 - val_accuracy: 0.9245
Epoch 4/20
50/50 [==============================] - 27s 547ms/step - loss: 0.0644 - accuracy: 0.9862 - val_loss: 0.4275 - val_accuracy: 0.9036
Epoch 5/20
50/50 [==============================] - 27s 546ms/step - loss: 0.0283 - accuracy: 0.9950 - val_loss: 0.3982 - val_accuracy: 0.9297
Epoch 6/20
50/50 [==============================] - 27s 547ms/step - loss: 0.0258 - accuracy: 0.9962 - val_loss: 0.3907 - val_accuracy: 0.9401
Epoch 7/20
50/50 [==============================] - 27s 546ms/step - loss: 0.0198 - accuracy: 0.9969 - val_loss: 0.3912 - val_accuracy: 0.9271
Epoch 8/20

*Примечание: ввиду стохастического характера процесса обучения нейросети результаты могут варьироваться от запуска к запуску. Инициализация всех псевдогенераторов случайных чисел (random.seed, numpy.random.seed, tensorflow.random.set_seed) данной проблемы не решает. На kagge были отправлены результаты прогноза модели, для которой val_accuracy после обучения составляла 0.9425*

Конкатенируем два блока - препроцессор и энкодер - в единую модель для прогнозирования тестовых данных.

In [ ]:
outputs = model(preprocessor(preprocessor.inputs))
trained_model = tf.keras.Model(preprocessor.inputs, outputs)

Загружаем тестовую выборку и вычисляем прогноз.

In [ ]:
%%time
test_data = pd.read_csv(fname_test, sep='\t')
test_dataset = tf.data.Dataset.from_tensor_slices(test_data['text'])
test_dataset = test_dataset.map(lambda x: [[x]])
predictions = []
for i, row in tqdm(enumerate(test_dataset)):
    # print(i, result, row[0].numpy()[0])
    predictions.append(tf.argmax(trained_model(row), axis=1)[0].numpy())

Cause: could not parse the source code of <function <lambda> at 0x7f9be2792710>: no matching AST found
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


500it [39:08,  4.70s/it]

CPU times: user 19.8 s, sys: 3.61 s, total: 23.4 s
Wall time: 39min 8s


Сохраняем прогноз в файл для отправки на kaggle.

In [ ]:
submission = pd.DataFrame({'y': predictions})
submission.index.name = 'Id'
submission.to_csv('submission.csv')